# Student Alcohol Consumption

### Introduction:

This time you will download a dataset from the UCI.

### Step 1. Import the necessary libraries

In [3]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
import requests

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/04_Apply/Students_Alcohol_Consumption/student-mat.csv).

### Step 3. Assign it to a variable called df.

In [8]:

url = 'https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/04_Apply/Students_Alcohol_Consumption/student-mat.csv'


get_url = requests.get(url)


with open('data.csv', 'w', encoding='utf-8') as f:
    
    f.write(get_url.text)
    

df = pd.read_csv('data.csv', header=0, sep=',')

spark = SparkSession.builder\
                    .appName('students')\
                    .getOrCreate()
                    
                    
students = spark.read.csv('data.csv', header=True, sep=',')

25/06/11 15:12:29 WARN Utils: Your hostname, kevin-llanos-Type1ProductConfigId resolves to a loopback address: 127.0.1.1; using 192.168.1.92 instead (on interface wlo1)
25/06/11 15:12:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/11 15:12:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/06/11 15:12:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


25/06/11 15:12:50 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


### Step 4. For the purpose of this exercise slice the dataframe from 'school' until the 'guardian' column

In [13]:

columnas = students.columns


inicio = columnas.index('school')
final = columnas.index('guardian')

subset_columnas = columnas[inicio:final]


students_subset = students.select(subset_columnas)


### Step 5. Create a lambda function that will capitalize strings.

### Step 6. Capitalize both Mjob and Fjob

### Step 7. Print the last elements of the data set.

### Step 8. Did you notice the original dataframe is still lowercase? Why is that? Fix it and capitalize Mjob and Fjob.

### Step 9. Create a function called majority that returns a boolean value to a new column called legal_drinker (Consider majority as older than 17 years old)

### Step 10. Multiply every number of the dataset by 10. 
##### I know this makes no sense, don't forget it is just an exercise